In [3]:
import sagemaker
import boto3

In [4]:
region = sagemaker.Session().boto_region_name

boto3.setup_default_session(region_name = region)
boto_session = boto3.Session(region_name = region)

s3_client = boto3.client("s3", region_name = region)

sagemaker_boto_client = boto_session.client("sagemaker")
sagemaker_session = sagemaker.session.Session(
    boto_session = boto_session, sagemaker_client = sagemaker_boto_client
)
account_id = boto3.client("sts").get_caller_identity()["Account"]
sagemaker_role = sagemaker.get_execution_role()

In [5]:
bucket = "BUCKET-NAME"
prefix = "mlops/predictive-maintenance"

### CSV Test Data with headers

In [6]:
import pandas as pd

In [7]:
test_data = pd.read_csv("datasets/train-test/test.csv")
test_data = test_data.drop("failure", axis = 1)
test_data.to_csv("datasets/train-test/test_data_with_headers.csv", index = False)

### Model Monitor Object

In [8]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

test_data_drift_monitor = DefaultModelMonitor(
    role = sagemaker_role,
    instance_count = 1,
    instance_type = "ml.m5.xlarge",
    volume_size_in_gb = 1,
    max_runtime_in_seconds = 360,
    sagemaker_session = sagemaker_session,
)

### Suggest Baselining

In [ ]:
from datetime import datetime

baseline_data = "datasets/train-test/test_data_with_headers.csv"
baseline_results_uri = f"s3://{bucket}/{prefix}/data/baselining/test-header-data-results"
baseline_job_name = f"PdM-Baseline-Job-Data-Monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"

test_data_drift_monitor.suggest_baseline(
    job_name = baseline_job_name,
    baseline_dataset = baseline_data,
    dataset_format = DatasetFormat.csv(header = True),
    output_s3_uri = baseline_results_uri,
)

In [19]:
baseline_job = test_data_drift_monitor.latest_baselining_job

In [21]:
schema_df = pd.io.json.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.name

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


0           voltmean_3h
1         rotatemean_3h
2       pressuremean_3h
3      vibrationmean_3h
4             voltsd_3h
5           rotatesd_3h
6         pressuresd_3h
7        vibrationsd_3h
8          voltmean_24h
9        rotatemean_24h
10     pressuremean_24h
11    vibrationmean_24h
12           voltsd_24h
13         rotatesd_24h
14       pressuresd_24h
15      vibrationsd_24h
16          error1count
17          error2count
18          error3count
19          error4count
20          error5count
21                comp1
22                comp2
23                comp3
24                comp4
25                  age
26         model_model1
27         model_model2
28         model_model3
29         model_model4
Name: name, dtype: object

### Creating Monitoring Schedule

In [14]:
preprocessor_path = f"s3://{bucket}/{prefix}/code/datacapture_preprocessing.py"

In [11]:
monitor_schedule_name = "PdM-DataDrift-Monitoring-Schedule-Header-Data"
endpoint_name = "PdM-SKLearn-Pipeline-Endpoint-ReTraining"

In [16]:
from sagemaker.model_monitor import CronExpressionGenerator

test_data_drift_monitor.create_monitoring_schedule(
    monitor_schedule_name = monitor_schedule_name,
    statistics = test_data_drift_monitor.baseline_statistics(),
    record_preprocessor_script = preprocessor_path,
    endpoint_input = endpoint_name,
    constraints = test_data_drift_monitor.suggested_constraints(),
    schedule_cron_expression = CronExpressionGenerator.hourly(),
    output_s3_uri = baseline_results_uri,
    enable_cloudwatch_metrics = True
)

INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: PdM-DataDrift-Monitoring-Schedule-Header-Data


In [9]:
test_data_drift_monitor.monitoring_schedule_name = "PdM-DataDrift-Monitoring-Schedule-Header-Data"

### Invoke Endpoint

In [36]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer, JSONSerializer

predictor = Predictor(
    endpoint_name = endpoint_name, 
    sagemaker_session = sagemaker_session,
    serializer = JSONSerializer()
)

In [73]:
import json
import time

for item in test_data.to_numpy()[:100]:
    item = [item.tolist()]
    result = predictor.predict(item)
    time.sleep(0.5)

In [71]:
type(result)

bytes

In [74]:
len(test_data_drift_monitor.list_executions())

2

In [89]:
test_data_drift_monitor.list_executions()[-1].describe()["ProcessingJobStatus"]

'Completed'

In [90]:
test_data_drift_monitor.list_executions()[-1].describe()["ExitMessage"]

'CompletedWithViolations: Job completed successfully with 1 violations.'

In [92]:
violations = test_data_drift_monitor.latest_monitoring_constraint_violations()
pd.set_option("display.max_colwidth", -1)
constraints_df = pd.io.json.json_normalize(violations.body_dict["violations"])
constraints_df.head(10)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,feature_name,constraint_check_type,description
0,Missing columns,missing_column_check,"There are missing columns in current dataset. Number of columns in current dataset: 3, Number of columns in baseline constraints: 30"


In [94]:
test_data.head()

,voltmean_3h,rotatemean_3h,pressuremean_3h,vibrationmean_3h,voltsd_3h,rotatesd_3h,pressuresd_3h,vibrationsd_3h,voltmean_24h,rotatemean_24h,...,error5count,comp1,comp2,comp3,comp4,age,model_model1,model_model2,model_model3,model_model4
0,170.301017,449.036995,94.805205,40.816797,11.061667,58.425055,4.931305,2.428740,176.844376,456.598107,...,0.0,28.875,13.875,118.875,28.875,18,0,0,1,0
1,165.339972,435.660354,103.351320,31.892462,10.717864,26.009485,22.071933,6.020669,176.141499,453.900566,...,0.0,29.000,14.000,119.000,29.000,18,0,0,1,0
2,183.752875,463.058640,109.525083,41.945037,9.369264,43.646584,10.859804,9.395067,175.764202,451.753148,...,0.0,29.125,14.125,119.125,29.125,18,0,0,1,0
3,177.866822,506.692032,98.745260,39.861149,16.596090,38.086352,10.410456,5.418325,175.352459,455.124136,...,0.0,29.250,14.250,119.250,29.250,18,0,0,1,0
4,167.471524,425.963281,111.996389,39.396999,9.015089,63.342755,4.648154,6.365146,174.712824,451.436346,...,0.0,29.375,14.375,119.375,29.375,18,0,0,1,0
